In [160]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

In [57]:
filename = "https://s3.amazonaws.com/name-gender/allnames.txt"
df=pd.read_csv(filename, sep=',', names = ["Name", "Gender", "Count"])


In [58]:
df.shape

(1891894, 3)

In [60]:
df.head(10)

,Name,Gender,Count
0,Mary,F,7065
1,Anna,F,2604
2,Emma,F,2003
3,Elizabeth,F,1939
4,Minnie,F,1746
5,Margaret,F,1578
6,Ida,F,1472
7,Alice,F,1414
8,Bertha,F,1320
9,Sarah,F,1288


In [63]:
df = df.drop(['Count'], axis=1)

In [64]:
df.head()

,Name,Gender
0,Mary,F
1,Anna,F
2,Emma,F
3,Elizabeth,F
4,Minnie,F


In [65]:
df = df.drop_duplicates()

In [66]:
df.shape

(106695, 2)

In [145]:
#number of names
num_names = df.shape[0]
print(num_names)
# length of longest name
max_name_length = (df['Name'].map(len).max())
print(max_name_length)

106695
15


In [75]:
print(df['Name'].map(len).min())

2


In [133]:
names = df['Name'].values
genders = df['Gender']

In [316]:
txt = ""
for n in names:
    txt += n.lower()
chars = set(txt)
alphabet_size = len(chars)
print('total chars:', len(chars))

total chars: 26


In [317]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print (char_indices)


{'b': 0, 'n': 1, 'y': 2, 'z': 3, 'v': 4, 'd': 5, 'a': 6, 'j': 7, 'h': 8, 't': 9, 'o': 10, 'w': 11, 'f': 12, 'k': 13, 'u': 14, 's': 15, 'c': 16, 'i': 17, 'r': 18, 'e': 19, 'm': 20, 'l': 21, 'g': 22, 'p': 23, 'x': 24, 'q': 25}


In [329]:
integer_encoded = [char_indices[char] for char in name]
X = np.zeros((num_names, max_name_length, alphabet_size))

for i,name in enumerate(names):
    name = name.lower()
    for t, char in enumerate(name):
        X[i, t,char_indices[char]] = 1
#for value in integer_encoded:
#    letter = [0 for _ in range(alphabet_size)]
#    letter[value] = 1
#    onehot_encoded.append(letter)

In [330]:
X[0,:,:]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [203]:
#X = np.ones((num_names, max_name_length,1), dtype=np.int64) * -1
#y = np.array(gender)

#for i,name in enumerate(names):
#    name = name.lower()
#    for t, char in enumerate(name[-max_name_length:]):
#        X[i, (max_name_length-1-t)] = char_indices[char]
#    print (X[i,:])

#for i,name in enumerate(names):
#    name = name.lower()
#    for t, char in enumerate(name):
#        X[i, t,0] = char_indices[char]
#    #print (X[i,:])


In [204]:
df['Gender'].value_counts()

F    66358
M    40337
Name: Gender, dtype: int64

In [213]:
#Y = np.where(df['Gender'].str.contains("F"), 1, other=-1)
Y = np.ones((num_names,1))
Y[df['Gender'] == 'F',0] = 0
Y

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

In [331]:
print (X.shape)
print (Y.shape)

(106695, 15, 26)
(106695, 1)


In [333]:
data_dim = alphabet_size
timesteps = max_name_length
num_classes = 1

In [483]:
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(1, activation='sigmoid'))

In [484]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [486]:
#model.fit(X, Y,
#          batch_size=64, epochs=5,
#          validation_data=(x_val, y_val))
model.fit(X, Y, validation_split=0.20, epochs=20, batch_size=64)

Train on 85356 samples, validate on 21339 samples
Epoch 1/20
85356/85356 [==============================] - 29s 336us/step - loss: 0.3791 - acc: 0.8274 - val_loss: 0.4394 - val_acc: 0.8021
Epoch 2/20
85356/85356 [==============================] - 29s 335us/step - loss: 0.3773 - acc: 0.8291 - val_loss: 0.4174 - val_acc: 0.8144
Epoch 3/20
85356/85356 [==============================] - 28s 327us/step - loss: 0.3756 - acc: 0.8291 - val_loss: 0.4309 - val_acc: 0.8035
Epoch 4/20
85356/85356 [==============================] - 28s 328us/step - loss: 0.3738 - acc: 0.8301 - val_loss: 0.4306 - val_acc: 0.8070
Epoch 5/20
85356/85356 [==============================] - 28s 330us/step - loss: 0.3718 - acc: 0.8321 - val_loss: 0.4282 - val_acc: 0.8078
Epoch 6/20
85356/85356 [==============================] - 29s 334us/step - loss: 0.3702 - acc: 0.8322 - val_loss: 0.4225 - val_acc: 0.8063
Epoch 7/20
85356/85356 [==============================] - 28s 327us/step - loss: 0.3686 - acc: 0.8339 - val_loss: 0.

In [481]:
num_test = 6
names_test = ['shahrukh', "rob", "victor","amaya","vetri", "swetha"]
X_test = np.zeros((num_test, max_name_length, alphabet_size))

for i,name in enumerate(names_test):
    name = name.lower()
    for t, char in enumerate(name):
        X_test[i, t,char_indices[char]] = 1

#X_test = np.ones((num_test, max_name_length,1), dtype=np.int64) * -1
#for i,name in enumerate(names_test):
#    name = name.lower()
#    for t, char in enumerate(name):
#        X_test[i, t,0] = char_indices[char]
predictions = model.predict(X_test)
predictions

array([[ 0.90565544],
       [ 0.78381199],
       [ 0.83999473],
       [ 0.03508203],
       [ 0.29466197],
       [ 0.05799102]], dtype=float32)

In [339]:
predictions.shape

(106695, 1)

array([[ 0.58957708],
       [ 0.2044933 ],
       [ 0.5969525 ],
       [ 0.20876125]], dtype=float32)